In [15]:
import json
import pandas as pd
import numpy as np
import time
import requests
import logging

In [16]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("economic-test")
logger.setLevel(logging.INFO)

In [36]:
def get_information(method, params):
#     url = 'https://api.s0.os.hmny.io/'
    url = 'https://api.s0.dry.hmny.io/'
#     url = 'https://api.s0.t.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [4]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [5]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [19]:
def getCurrentAndLastBlock():
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    print("current and last block numbers", block, last_block)
    return block, last_block

In [34]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result'] 

In [24]:
def getAvailabilityAndRewards():
    reward = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['current-epoch-performance']:
            sign = i['current-epoch-performance']['current-epoch-signing-percent']
            if sign['current-epoch-to-sign'] == 0:
                continue
            perc = float(sign['current-epoch-signing-percentage'])
            if perc > 2/3:
                address = i['validator']['address']
                reward_accumulated = i['lifetime']['reward-accumulated']
                reward[address] = reward_accumulated
    return reward

In [8]:
def getRewardsAndStatus(cutoff):
    reward = dict()
    status = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        address = i['validator']['address']
        if address in cutoff: 
            reward_accumulated = i['lifetime']['reward-accumulated']
            reward[address] = reward_accumulated
            epos_status = i['epos-status']
            status[address] = epos_status
    return reward, status


In [20]:
def CN1_test(single):
    logger.info(f"Test-CN1: Slow validator is never starved (should be able to sign blocks)")
    curr_test = None
    
    block, last_block = getCurrentAndLastBlock()
    logger.info(f"current and last block numbers: {block}, {last_block}")
    if block == last_block or block == last_block -1:
        logger.info(f"current at the last block or last second block, wait until the 5th block in the new epoch")
        while block < last_block+5:
            block = getBlockNumber()
    logger.info(f"current block: {block}, will begin collecting infos...")
    # get the validator's reward who just meets the 2/3 cut-off  
    cutoff_rewards = getAvailabilityAndRewards()
    if not cutoff_rewards:
        logger.info(f"no validators sign more than 2/3 in this test\n")
        return "Need More Tests", curr_test
    new_block = block + 1
    while block < new_block:
        block = getBlockNumber()
    logger.info(f"new block reached, {block}, will wait for 5 seconds to begin testing")
    time.sleep(5)
    next_rewards, status = getRewardsAndStatus(cutoff_rewards)
    flag = True
    for k,v in next_rewards.items():
        reward_per_block = v - cutoff_rewards[k]
        if reward_per_block == 0 or status[k] == 'not eligible to be elected next epoch':
            flag = False
            logger.warning(f"Test-CN1: Fail")
            if reward_per_block == 0:
                logger.warning(f"Slow validator {k} doesn't get reward\n")
            if status[k] == 'not eligible to be elected next epoch':
                logger.warning(f"Slow validator {k} is no longer eligible\n")

    if flag:
        logger.info(f"Test-CN1: Succeed\n")
        return True, curr_test
    else:
        return False, curr_test

In [30]:
CN1_test(True)

INFO:economic-test:Test-CN1: Slow validator is never starved (should be able to sign blocks)
INFO:economic-test:current and last block numbers: 3224119, 3224269
INFO:economic-test:current block: 3224119, will begin collecting infos...


current and last block numbers 3224119 3224269


INFO:economic-test:new block reached, 3224120, will wait for 5 seconds to begin testing
INFO:economic-test:Test-CN1: Succeed



(True, None)

In [25]:
CN1_test(True)

INFO:economic-test:Test-CN1: Slow validator is never starved (should be able to sign blocks)
INFO:economic-test:current and last block numbers: 3223912, 3224269
INFO:economic-test:current block: 3223912, will begin collecting infos...


current and last block numbers 3223912 3224269


INFO:economic-test:new block reached, 3223913, will wait for 5 seconds to begin testing







(False, None)

In [10]:
def getValidatorInfo(validator):
    method = "hmy_getValidatorInformation"
    params = [validator]
    return get_information(method, params)['result']

In [26]:
i = 'one103q7qe5t2505lypvltkqtddaef5tzfxwsse4z7'
getValidatorInfo(i)

{'validator': {'bls-public-keys': ['86dc2fdc2ceec18f6923b99fd86a68405c132e1005cf1df72dca75db0adfaeb53d201d66af37916d61f079f34f21fb96'],
  'last-epoch-in-committee': 182,
  'min-self-delegation': 10000000000000000000000,
  'max-total-delegation': 12600000000000000000000000000,
  'rate': '0.050000000000000000',
  'max-rate': '1.000000000000000000',
  'max-change-rate': '0.050000000000000000',
  'update-height': 3221644,
  'name': 'Rongjian',
  'identity': 'rj',
  'website': 'rongjian@harmony.one',
  'security-contact': 'harmony',
  'details': 'RJ the validator',
  'creation-height': 3221644,
  'address': 'one103q7qe5t2505lypvltkqtddaef5tzfxwsse4z7',
  'delegations': [{'delegator-address': 'one103q7qe5t2505lypvltkqtddaef5tzfxwsse4z7',
    'amount': 10000000000000000000000,
    'reward': 1066139721385542171141,
    'undelegations': []},
   {'delegator-address': 'one18xfcqu7jf0cq5apweyu5jxr30x9cvetegwqfss',
    'amount': 9700000000000000000000,
    'reward': 178540625000000001426,
    'unde

In [27]:
i = 'one18vn078vyp5jafma8q7kek6w0resrgex9yufqws'
getValidatorInfo(i)

{'validator': {'bls-public-keys': ['394d726edbb908fa71bdb2b417dd0fe7c77d9f86b3a0b1e1b51d11e26dd1cddb4e21269c3e1d08e75d43ec98ff4ca582'],
  'last-epoch-in-committee': 182,
  'min-self-delegation': 15000000000000000000000,
  'max-total-delegation': 10000000000000000000000000,
  'rate': '0.050000000000000000',
  'max-rate': '0.500000000000000000',
  'max-change-rate': '0.050000000000000000',
  'update-height': 3221765,
  'name': 'leo-130',
  'identity': 'Amazonian!',
  'website': 'https://staking.leochen.net',
  'security-contact': 'staking@leochen.net',
  'details': 'Amazon Veteran with strong DevOps experience',
  'creation-height': 3221765,
  'address': 'one18vn078vyp5jafma8q7kek6w0resrgex9yufqws',
  'delegations': [{'delegator-address': 'one18vn078vyp5jafma8q7kek6w0resrgex9yufqws',
    'amount': 15000000000000000000000,
    'reward': 15721889355329633964432,
    'undelegations': []}]},
 'current-epoch-performance': {'current-epoch-signing-percent': {'current-epoch-signed': 103,
   'cur

In [28]:
i = 'one1gjsxmewzws9mt3fn65jmdhr3e4hel9xza8wd6t'
getValidatorInfo(i)

{'validator': {'bls-public-keys': ['084c40353ede41cdd52b7b142884269a324022ce21826c9ecae66f6acf8de2c9bc9d1fae2f87a31ba0a0a1863a52f506',
   '73aab123084e79be660e6e3a2102e1087b3db3a762d2fc4f0f5d7a7b99d326a97490e75a548e7a1f5d451757a331ae0e',
   '93ddd28639e0878547134f4fb4cd26819042d76969ba520adb7a566a693b076c45e348b2a26876154af847a1b30fa492',
   'a15560ec4d62de1a8b8e030445e30e644afdb3b4c1e5f06db95cc26050e05ce8ada141d04038e31abbc3ac0b141c860e'],
  'last-epoch-in-committee': 182,
  'min-self-delegation': 10000000000000000000000,
  'max-total-delegation': 100000000000000000000000000,
  'rate': '0.075000000000000000',
  'max-rate': '0.750000000000000000',
  'max-change-rate': '0.100000000000000000',
  'update-height': 3221955,
  'name': 'JB273-OpenStaking #P-OPS',
  'identity': 'JB273',
  'website': 'JB273.harmony.one',
  'security-contact': 'JB273',
  'details': 'OpenStaking made possible by JB273',
  'creation-height': 3221955,
  'address': 'one1gjsxmewzws9mt3fn65jmdhr3e4hel9xza8wd6t',
  'de